In [26]:
#tensor flow guide book https://archive.org/details/ai-ml/page/24/mode/2up
import tensorflow as tf # for ML and we will be using a tensorflow data set/ the mnist number data set
#tensorflow guide https://www.tensorflow.org/guide/keras/sequential_model
import numpy as np #working with arrays
import matplotlib.pyplot as plt # for visualization
import cv2 #for computer vision
import os


In [27]:
_data = tf.keras.datasets.mnist #this is a training dataset generated by MNIST for number recoginition first data set made by mnist
#due to this data being already split up in test adn training data we can call this below without splitting or data preprocessing
(x_train, y_train), (x_test, y_test)= _data.load_data() #x is the pixel data, y is the classification/number



When we train our own data
using sklearn

from sklearn.model_selection import train_test_split

 Assuming you have your data in variables x (images) and y (labels)

 Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

 test_size=0.2 indicates that 20% of the data will be used for testing
 random_state is used to ensure reproducibility, setting it to a fixed value ensures the same random split each time you run the code

for the datapipeline it would probably benefit from using tf.datasets framework for images 
pandas for statistical data 
refer to this page:
https://www.tensorflow.org/guide/data


In [28]:
#now lets normalize the data/grey scaling 
"""gray scale pixel has value of  255  normalizing would make it between 0 and 1 
it will be scaling things down the value of 125 might look like 0.5"""

"""axis=-1:
When axis=-1, normalization is performed along the last axis of the input data.
In the context of a 2D tensor (like a matrix), where the first axis corresponds to rows and the second axis corresponds to columns, axis=-1 implies normalization is performed along the columns.
This means that each feature (column) in the input data is independently normalized across the samples.
axis=1:
When axis=1, normalization is performed along the second axis of the input data.
In the context of a 2D tensor, where the first axis corresponds to rows and the second axis corresponds to columns, axis=1 implies normalization is performed along the rows.
This means that each sample (row) in the input data is independently normalized across its features.

Normalization Along Rows:
With axis=0, normalization is applied along each feature (column) across all the samples (rows).
Each feature's mean and standard deviation are calculated across all the samples, considering all the values of that feature across all samples.
The normalization is then applied to each sample's feature values based on these aggregated statistics.

"""
x_train= tf.keras.utils.normalize(x_train, axis= 1)
x_test = tf.keras.utils.normalize(x_test, axis =1)

In [29]:
#check the shape of what x_train and x_test is
print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

Training data shape: (60000, 28, 28)
Test data shape: (10000, 28, 28)


In [30]:
# Plot some of the images from the training set
"""
plt.figure(figsize=(10, 10))
for i in range(25):  # Plotting the first 25 images
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.title(f'Label: {y_train[i]}')  # Show the label as the title
plt.show()
"""

"\nplt.figure(figsize=(10, 10))\nfor i in range(25):  # Plotting the first 25 images\n    plt.subplot(5, 5, i + 1)\n    plt.imshow(x_train[i], cmap='gray')\n    plt.axis('off')  # Turn off axis labels\n    plt.title(f'Label: {y_train[i]}')  # Show the label as the title\nplt.show()\n"

In [31]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28,28))) #flattens the data turns the pixels from 28x28 to a line between 784
#flatten turns it from 2-D to a 1-D array

#add relu layer and softmax/optimization functions below:
model.add(tf.keras.layers.Dense(128,activation = tf.nn.relu)) #model.add(tf.keras.layers.Dense(128,activation = 'relu')) relu follows something simlar to
#sigmoid activation relu = f(x)=max(0,x) It returns 0 for all negative inputs and returns the input value for all positive inputs.

#relu is the activation function in the dense layer and 128 is the number of units
model.add(tf.keras.layers.Dense(128,activation = tf.nn.relu)) # make a second layer 
model.add(tf.keras.layers.Dense(10,activation = tf.nn.softmax)) # now the softmax layer all the 10 neurons will add up to one 
#softmax is the output layer

In [ ]:
"""
#how relu works:
import numpy as np

# Define a sample input array
input_array = np.array([-2, -1, 0, 1, 2])

# Apply ReLU activation function
output_array = np.maximum(0, input_array)

print("Input array:", input_array)
print("Output array (ReLU):", output_array)

#Input array: [-2 -1  0  1  2]
#Output array (ReLU): [0 0 0 1 2]


#how softmax works:
import numpy as np

def softmax(logits):
    exp_logits = np.exp(logits)
    softmax_scores = exp_logits / np.sum(exp_logits)
    return softmax_scores

# Example logits (raw scores)
logits = np.array([2.0, 1.0, 0.1])

# Compute softmax probabilities
probabilities = softmax(logits)

print("Raw Scores (Logits):", logits)
print("Softmax Probabilities:", probabilities)
print("Sum of Probabilities:", np.sum(probabilities))

#Raw Scores (Logits): [2.  1.  0.1]
#Softmax Probabilities: [0.65900114 0.24243297 0.09856589]
#Sum of Probabilities: 1.0
"""


In [38]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#uses adam optimization make sure you spell the loss function right

Cross-entropy is a concept from information theory and is commonly used as a loss function in machine learning, particularly in classification tasks. Mathematically, cross-entropy measures the difference between two probability distributions: the predicted probability distribution and the true probability distribution.

Given:

p(y): The true probability distribution of the classes. For example, in binary classification,  p(y) could be a one-hot encoded vector representing the true class label.

q(y): The predicted probability distribution of the classes. For example, output of a softmax function representing predicted class probabilities.

The cross-entropy between 
p and q, denoted as H(p,q), is defined as:
H(p, q) = -\sum_{i} p(y_i) \log(q(y_i))
Where:

i iterates over all classes,

p(y_i)  is the true probability of class i

q(y_i) is the predicted probability of class i

In binary classification, where  p(y) is a one-hot encoded vector with the true class label, the cross-entropy simplifies to the following form:
H(p, q) = -[p(y) \log(q(y)) + (1 - p(y)) \log(1 - q(y))]



In [39]:
#fit the model pass the training data

model.fit(x_train,y_train, epochs=3) 
# epochs is how many iterations the model will see the data

#you can use clusting nearest neighbor instead / convolutional neural netoworks or

model.save('handwritten.model')

Epoch 1/3



1875/1875 [==============================] - 2s 958us/step - loss: 0.0583 - accuracy: 0.9816
Epoch 2/3
1875/1875 [==============================] - 2s 955us/step - loss: 0.0435 - accuracy: 0.9858
Epoch 3/3
1875/1875 [==============================] - 2s 960us/step - loss: 0.0318 - accuracy: 0.9895
INFO:tensorflow:Assets written to: handwritten.model\assets


INFO:tensorflow:Assets written to: handwritten.model\assets


In [40]:
model = tf.keras.models.load_model('handwritten.model')
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
#low loss and high accuracy is better 
#will write to a .pb file with the name of handwritten model 
#pb stands for proto buffer
# protobuffer is data type like json

313/313 [==============================] - 0s 847us/step - loss: 0.1020 - accuracy: 0.9730
Test Loss: 0.10195577144622803
Test Accuracy: 0.9729999899864197


In [ ]:
#use os stream and paint to test this model


image_number = 0
while os.path.is(f"foldername/label{image_number}.png"):
    try:
        img = cv2.imread(f"Imag{image_number}.png")[::0]
        #img = cv2.imread(f"foldername/label{image_number}.png")[::0]#we dont care about colors so [:,:,0] will be white on black inverted
        #need to invert it to be black on white use numpy to invert the numbers
        img = np.invert(np.array([img])) 
        #img will be an array of a list of img
        prediction = model.predict(img)
        print(f"the number is probably {np.argmax(prediction)}") #argmax gives index of the field of with the highest number which neuron has the highest activation
        #zeroth with represent zero the first will represent 1
        plt.imshow(img[0],cmap=plt.cm.binary)
        plt.show()
    except:
        print error
    finally: #nomatter what this will execute
        image_number+=1

#if model doesn't work well train with more epoch
